# Final Deliverable

## Business Problem

I am an immigrant in Canada from India with a cooking expertise. I want to open a food-chain focusing on Indian cuisine in Toronto. For the starting of the business, I need some suggestion regarding the neighborhood of Toronto. In this regard, I have two options: being more competitive or being for innovative. If I want to go competitive then I want to locate my first restaurant to be in the center where there are a lot of highly rated Indian restaurant. Else I want to locate my restaurant somewhere nearby where Indian cuisines are not available or not that much highly rated. I need both kind of suggestion in terms of location.

## Data

We will be using Foursquare API to explore a geographical location. We will be using this to locate Indian restaurants. We will also ne using Foursquare API for the user rating of those places. And for the latitude and longitude we will be using the file provided in http://cocl.us/Geospatial_data.

### Importing necessary library

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import numpy as np

### User Credentials

In [3]:
CLIENT_ID = 'MFJLBG2F0TYVVTI3DHZASIBPL2AUB5AW3GCMSY4ZZR0UTL4U' # your Foursquare ID
CLIENT_SECRET = 'OVPAA3IQLB45V3LRO1H0TMFP1RBUMV4XFPC40DHC2MALLI5X' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MFJLBG2F0TYVVTI3DHZASIBPL2AUB5AW3GCMSY4ZZR0UTL4U
CLIENT_SECRET:OVPAA3IQLB45V3LRO1H0TMFP1RBUMV4XFPC40DHC2MALLI5X


### Loading location data with postal codes in Toronto

In [4]:
location_data = pd.read_csv("http://cocl.us/Geospatial_data")
location_data.columns=['PostalCode','Latitude','Longitude']
location_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Example-Search for first geographical coordinate for "Indian" cuisine in 3Km radius

In [19]:
search_query = 'Indian'
radius = 3000

In [20]:
latitude=location_data.iloc[0,1]
longitude=location_data.iloc[0,2]
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [21]:
results = requests.get(url).json()

In [22]:
# assign relevant part of JSON to venues
venues = results['response']['venues']
# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d144941735', 'name': 'C...",False,4d2c8332342d6dcbe82020cb,1154 Morningside Ave,CA,Scarborough,Canada,787,"[1154 Morningside Ave, Scarborough ON M1B 3A4,...","[{'label': 'display', 'lat': 43.8005806, 'lng'...",43.800581,-79.199309,NaN,M1B 3A4,ON,Charley's West Indian Foods Ltd,v-1591956857
1,"[{'id': '58daa1558bbb0b01f18ec1fd', 'name': 'Z...",False,4cd8637d145fa1cdda2b06bd,NaN,CA,Toronto,Canada,1738,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.81926565890734...",43.819266,-79.181535,Rouge,NaN,ON,Indian Rhinoceros Pavilion,v-1591956857
2,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4e35b8a8ae60d86c3ac2e8d4,31 Tapscott Rd. Unit B2,CA,Toronto,Canada,2159,"[31 Tapscott Rd. Unit B2, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.80746450749075...",43.807465,-79.221210,NaN,NaN,ON,Rubini West Indian Food Market,v-1591956857


### Gettting the venue rating

In [30]:
# For the first place
venue_id = dataframe.id[0]
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

In [31]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.
